In [ ]:
!pip install transformers pandas numpy scikit-learn tensorflow -q

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import os

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
def load_and_validate_data(file_path="/content/drive/My Drive/Project and Coding/BERT AND ELECTRA/Apple_tweets_setiment.csv"):
    try:
        df = pd.read_csv(file_path, on_bad_lines='skip', quoting=3)
        if 'tweets' not in df.columns or 'labels' not in df.columns:
            raise ValueError("Dataset must contain 'tweets' and 'labels' columns")
        df = df.dropna(subset=['tweets', 'labels'])

        # Filter out neutral (0.0) labels. We only do for binary sentiment
        df = df[df['labels'] != 0.0]

        df['labels'] = df['labels'].apply(lambda x: 0 if x == -1.0 else 1)

        print("Dataset Info:")
        print(df.info())
        print("\nSample Data:")
        print(df.head())
        print("\nLabel Distribution:")
        print(df['labels'].value_counts())

        return df

    except FileNotFoundError:
        print("Error: Dataset file not found.")
        return None


In [ ]:
df=load_and_validate_data()
if df is None:
  raise SystemExit("Failed to load dataset. Exiting.")

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['tweets'],
    df['labels'],
    test_size=0.2,
    random_state=42,
    stratify=df['labels']
)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Tokenize data
def tokenize_data(texts, max_length=128):
    return tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='tf'
    )

train_encodings = tokenize_data(train_texts)
test_encodings = tokenize_data(test_texts)

In [ ]:
def create_dataset(encodings, labels):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
            'token_type_ids': encodings['token_type_ids']
        },
        labels
    ))
    return dataset

train_dataset = create_dataset(train_encodings, tf.convert_to_tensor(list(train_labels)))
test_dataset = create_dataset(test_encodings, tf.convert_to_tensor(list(test_labels)))

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy('accuracy'),
    tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='top_1_accuracy')
]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=1,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/Project and Coding/BERT AND ELECTRA/best_bert_model_2',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )
]

In [ ]:
print("\nTraining BERT model...")
history = model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=3,
    batch_size=16,
    validation_data=test_dataset.batch(16),
    callbacks=callbacks
)

In [ ]:
print("\nEvaluating model...")
predictions = model.predict(test_dataset.batch(16))
pred_labels = np.argmax(predictions.logits, axis=1)

In [ ]:
print("\nClassification Report:")
print(classification_report(test_labels, pred_labels, target_names=['Negative', 'Positive']))
print("Accuracy:", accuracy_score(test_labels, pred_labels))
print("F1 Score:", f1_score(test_labels, pred_labels))

In [ ]:
save_path = "/content/drive/MyDrive/Project and Coding/BERT AND ELECTRA/bert_sentiment_model_2"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"\nModel saved to {save_path}")

In [ ]:
# Prediction function
def predict_sentiment(text):
    try:
        inputs = tokenizer(
            text,
            return_tensors="tf",
            truncation=True,
            padding=True,
            max_length=128
        )
        outputs = model(inputs)
        probs = tf.nn.softmax(outputs.logits, axis=-1)
        pred = np.argmax(probs, axis=1)[0]
        confidence = probs[0][pred].numpy()
        return ("Positive" if pred == 1 else "Negative", confidence)
    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        return None, None

In [ ]:
sample_texts = [
    "Absolutely love the new iPhone! Apple nailed it again.",
    "Apple stock is dropping after the disappointing earnings report.",
    "Thinking of switching to Android, tired of Apple's constant price hikes.",
    "The new iOS update is so smooth and fast. Great job Apple!",
    "Apple's customer service was incredibly helpful today.",
    "My MacBook just crashed again, so frustrating!",
    "Apple Pay makes checkout so much easier. Love this feature!",
    "Battery life on my iPhone is terrible after the latest update.",
    "AirPods Pro sound amazing! Totally worth the price.",
    "Apple Watch saved my life by detecting my irregular heartbeat."
]

print("\nExample Predictions:")
for text in sample_texts:
    sentiment, confidence = predict_sentiment(text)
    print(f"Text: {text}")
    if sentiment is not None and confidence is not None:
        print(f"Sentiment: {sentiment}, Confidence: {confidence:.4f}\n")
    else:
        print(f"Error predicting sentiment for text: {text}\n")